In [1]:
import sys
import numpy as np
import gym
from matplotlib import pyplot as plt
from dqn import DQN
from drqn import DRQN, Memory
from reinforce import REINFORCE, PiApproximationWithNN, Baseline
from reinforce_Buffer import REINFORCE as RF_Buffer, PiApproximationWithNN as Pi_Buffer, ReplayMemory

def test_DQN(env, run):
    gamma = 1.0
    return DQN(env, gamma, 500, run)


def test_DRQN(env, run):
    gamma = 1.0
    return DRQN(env, gamma, 500, run)

def test_reinforce(env,runs):
    gamma = 1.
    alpha = 3e-4

    if 'tensorflow' in sys.modules:
        import tensorflow as tf
        tf.reset_default_graph()

    pi = PiApproximationWithNN(
        env.observation_space.shape[0],
        env.action_space.n,
        alpha)

    B = Baseline(0.)

    return REINFORCE(env, gamma, 1000, runs, pi, B)


def test_reinforce_Buffer(env, mem_size, runs):
    gamma = 1.
    alpha = 3e-4

    if 'tensorflow' in sys.modules:
        import tensorflow as tf
        tf.reset_default_graph()

    pi = Pi_Buffer(
        env.observation_space.shape[0],
        env.action_space.n,
        alpha,
        mem_size)

    B = Baseline(0.)

    return RF_Buffer(env, gamma, 1000, runs, pi, B, mem_size)




/home/scarr/ReinforcementLearning/rl_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/scarr/ReinforcementLearning/rl_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/scarr/ReinforcementLearning/rl_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/scarr/ReinforcementLea

In [2]:
num_iter = 1
env = gym.make("CartPole-v0")

without_buffer = []
for q in range(num_iter):
    print('***************************************')
    print("----------------> Without Buffer: {}".format(q))
    print('***************************************')
    training_progress = test_reinforce(env,q)
    without_buffer.append(training_progress[0])
    pi = training_progress[1]
without_buffer = np.mean(without_buffer, axis=0)
    # play(env,pi)
    
    # Test REINFORCE_buffer size 2 and 5
with_buffer2 = []
for q in range(num_iter):
    print('***************************************')
    print("----------------> With Buffer = 2: {}".format(q))
    print('***************************************')
    training_progress = test_reinforce_Buffer(env, 2, q)
    with_buffer2.append(training_progress[0])
    pi_buff = training_progress[1]
with_buffer2 = np.mean(with_buffer2, axis=0)
    

***************************************
----------------> Without Buffer: 0
***************************************

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.random.categorical` instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





AttributeError: 'ImageData' object has no attribute 'data'

In [ ]:
# Plot the experiment result
fig,ax = plt.subplots()
ax.plot(np.arange(len(without_buffer)), without_buffer, label='No Buffer')
ax.plot(np.arange(len(with_buffer2)), with_buffer2, label='Buffer - Size 2')
# ax.plot(np.arange(len(with_buffer5)), with_buffer5, label='Buffer - Size 5')

ax.set_xlabel('iteration')
ax.set_ylabel('G_0')
ax.legend()

plt.show()


In [ ]:
# # Test DQN
dqn_list = []
dqn_policies = []
for q in range(num_iter):
    dqn_rew, dqn_pi = test_DQN(env, q)
    dqn_list.append(dqn_rew)
    dqn_policies.append(dqn_pi)
dqn_result = np.mean(dqn_list,axis=0)
smoothed_dqn_result = running_mean(dqn_result, 10)
#
# Test DRQN
drqn_list = []
drqn_policies = []
for q in range(num_iter):
    drqn_rew, drqn_pi = test_DRQN(env, q)
    drqn_list.append(drqn_rew)
    drqn_policies.append(drqn_pi)
drqn_result = np.mean(drqn_list, axis=0)
smoothed_drqn_result = running_mean(drqn_result, 10)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(len(smoothed_dqn_result)), smoothed_dqn_result, label='DQN_smoothed')
ax.plot(np.arange(len(dqn_result)), dqn_result, label='DQN', color='red', alpha=0.3)
ax.plot(np.arange(len(smoothed_drqn_result)), smoothed_drqn_result, label='DRQN_smoothed')
ax.plot(np.arange(len(drqn_result)), drqn_result, label='DRQN', color='grey', alpha=0.3)

ax.set_xlabel('iteration')
ax.set_ylabel('G_0')
ax.legend()

plt.show()
